Retrieval-Augmented Generation (RAG) Setup

In [1]:
# Activate virtual Python environment
#! source ../venv/bin/activate

Environment setup

In [ ]:
import os
from dotenv import load_dotenv

from langchain_elasticsearch import ElasticsearchStore
from langchain_aws import BedrockEmbeddings
from langchain_aws import BedrockLLM
from langchain.chains import RetrievalQA
from urllib.request import urlopen

import boto3
import json


# Load all environment variables from .env file
load_dotenv()

ELASTIC_CLOUD_ENDPOINT = os.getenv('ELASTIC_CLOUD_ENDPOINT')
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_KEY')
AWS_REGION = os.getenv('AWS_REGION')
ELASTIC_CLOUD_ID = os.getenv('ELASTIC_CLOUD_ID')
bedrock_client = boto3.client(
    service_name = "bedrock-runtime",
    region_name = AWS_REGION
)

bedrock_embedding = BedrockEmbeddings(client=bedrock_client)


Create Elasticsearchstore

In [12]:
bedrock_embedding = BedrockEmbeddings(client=bedrock_client)

vector_store = ElasticsearchStore(
    es_url=ELASTIC_CLOUD_ENDPOINT,
    es_api_key=ELASTIC_API_KEY,
    index_name="marketplace_index",
    embedding=bedrock_embedding,
)

AuthenticationException: AuthenticationException(401, 'security_exception', 'unable to authenticate with provided credentials and anonymous access is not allowed for this request')